In [27]:
import numpy as np
import random
import copy
import sys

# ↓問題生成

In [54]:
def puzzle_generation(puzzle_board, issue):
    #ファイル読み込み
    f = open(issue, 'r', encoding='UTF-8')
    data = f.readlines()
    f.close()


    #問題フォーマットの1行目の読み取り ----------------------------------------------------------------------------------------
    s0 = data[0].rstrip('\n').rstrip(',')
    l0 = [int(x.strip()) for x in s0.split(',')]

    colmun = l0[0]              #列
    row = l0[1]                 #行
    selection_cost = l0[2]      #選択コスト


    
    #スライドパズルの盤面の読み取り ------------------------------------------------------------------------------------------
    # for i in range(1, row+1):
    #     row_data = data[i].rstrip('\n').rstrip(',')
    #     line_data = [int(x.strip()) for x in row_data.split(',')]
    #     row_array = np.array(line_data)
    #     if i == 1:
    #         puzzle_board = row_array
    #     else:
    #         puzzle_board = np.vstack((puzzle_board, row_array))

    # colmun = 5
    # row = 6
    #ランダム生成 ---------------------------------------------------------------------------------------------------------
    random_num = random.sample(range(1, row*colmun+1), row*colmun)
    random_list = np.array(random_num)
    puzzle_board = random_list.reshape(row, colmun)


    return puzzle_board, colmun, row, selection_cost

# ↓移動関数

In [55]:
#選択画像を上に移動
def up(target_value, selection_value, exchange_operation, fixed_value):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    if selection_coord[0] != 0 and (puzzle_board[selection_coord[0]-1][selection_coord[1]] in fixed_value) == False:
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]-1][selection_coord[1]])
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]-4], selection_value)

        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
        selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

        exchange_operation += 'U'

        return target_coord, selection_coord, exchange_operation

    
    else:
        return sys.exit("この上は盤面外、もしくは並び替えが完了しているよ!")

In [56]:
#選択画像を下に移動
def down(target_value, selection_value, exchange_operation, fixed_value):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    if selection_coord[0] != 3 and (puzzle_board[selection_coord[0]+1][selection_coord[1]] in fixed_value) == False:
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]+1][selection_coord[1]])
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]+4], selection_value)

        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
        selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

        exchange_operation += 'D'

        return target_coord, selection_coord, exchange_operation

    else:
        return sys.exit("この下は盤面外、もしくは並び替えが完了しているよ!")

In [57]:
#選択画像を左に移動
def left(target_value, selection_value, exchange_operation, fixed_value):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    if selection_coord[1] != 0 and (puzzle_board[selection_coord[0]][selection_coord[1]-1] in fixed_value) == False:
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]][selection_coord[1]-1])
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]-1], selection_value)

        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
        selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

        exchange_operation += 'L'

        return target_coord, selection_coord, exchange_operation

    else:
        return sys.exit("この左は盤面外、もしくは並び替えが完了しているよ!")

In [58]:
#選択画像を右に移動
def right(target_value, selection_value, exchange_operation, fixed_value):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    if selection_coord[1] != 3 and (puzzle_board[selection_coord[0]][selection_coord[1]+1] in fixed_value) == False:
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]][selection_coord[1]+1])
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]+1], selection_value)

        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
        selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

        exchange_operation += 'R'

        return target_coord, selection_coord, exchange_operation

    else:
        return sys.exit("この右は盤面外、もしくは並び替えが完了しているよ!")

# ↓回答書き込み

In [59]:
def answer_writing(selection_position, exchange_times, exchange_operation):
    f = open('answer.txt', 'w')
    f.write(str(selection_position[0]) + str(selection_position[1]) + '\n')
    f.write(str(exchange_times) + '\n')
    f.write(exchange_operation + '\n')    
    f.close()

    exchange_operation = ""

    return exchange_operation

# ↓それぞれの処理

In [60]:
# 2行を除いた盤面の並び替え(たくと)
def puzzle_board_sort_top(puzzle_board, selection_times, exchange_operation, fixed_value):
    #選択値の初期位置を記憶させておく
    selection_value = colmun * row
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))
    selection_position = selection_coord                #選択画像初期位置

    for i in range(row - 2):
        for j in range(colmun):
            target_value = i * colmun + j + 1           #動かしたい値
            selection_value = colmun * row              #選択値

            target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
            selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

            print("target_num = " + str(target_value) + "----------------------------------------------------------")
            
            #最終的に置きたい座標
            if target_value % colmun == 0: coord = (target_value // colmun - 1, colmun - 1)
            else: coord = (target_value // colmun, target_value % colmun - 1)
            
            #targetが下端でない
            if target_coord[0] != row-1:
                #最終的に起きたい位置が右2列以外
                if j != colmun-1 and j != colmun-2:
                    print("not下端, 右2列以外")

                    #targetが最終的に置きたいx座標よりも右
                    if target_coord[1] > coord[1]:
                        #targetの左下にchoice(16)を置く
                        if target_coord[1] <= selection_coord[1]:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        #targetを最終的に置きたいy座標に移動
                        if target_coord[0] < selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                        #targetを置きたい座標のx座標と同じになるまで移動
                        if target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            while target_coord[1] != coord[1]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                    
                    #targetを最終的に置きたいx座標よりも左
                    elif target_coord[1] < coord[1]:
                        #targetの右下にchoice(16)を置く
                        if target_coord[1] >= selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[0]+1 != selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        #targetを置きたい座標のx座標と同じになるまで移動
                        if target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            while target_coord[1] != coord[1]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            #targetの右にchoice(16)をおく
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        
                    
                    #targetが最終的に置きたいx座標と同じ
                    elif target_coord[1] == coord[1]:
                        #targetの右にchoice(16)をおく
                        #targetよりchoiceが右
                        if target_coord[1] < selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりchoiceが上
                            if target_coord[0] > selection_coord[0]:
                                while target_coord[0] != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりchoiceが下
                            else:
                                while target_coord[0] != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)

                        #targetよりchoiceが左
                        elif target_coord[1] > selection_coord[1]:
                            #targetがchoiceより上or下
                            if target_coord[0] != selection_coord[0]:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                #targetよりchoiceが上
                                if target_coord[0] > selection_coord[0]:
                                    while target_coord[0] != selection_coord[0]:
                                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                #targetよりchoiceが下
                                else:
                                    while target_coord[0] != selection_coord[0]:
                                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            #targetがchoiceと同じx座標
                            else:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        
                        #targetとchoiceのx座標が同じ
                        else:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりchoiceが上
                            if target_coord[0] > selection_coord[0]:
                                while target_coord[0] != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりchoiceが下
                            else:
                                while target_coord[0] != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)

                         
                    #targetを置きたい座標のy座標と同じになるまで移動
                    if target_coord[0] != coord[0]:
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[0] != coord[0]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                
                #最終的に置きたい位置が右から2列目------------------------------------------------------------------------------------------------------------------
                elif j == colmun-2:
                    print("not下端, 右から2列目")

                    #最終的に置きたい座標にある
                    if target_coord[0] == coord[0] and target_coord[1] == coord[1]:
                        while target_coord[1]+1 != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[0] != selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                    #最終的に置きたいy座標でない
                    elif target_coord[0] != coord[0]:
                        #最終的におきたいx座標でない
                        if target_coord[1] != coord[1]+1:
                            #targetよりも下にchoiceがある
                            if target_coord[0] < selection_coord[0]:
                                while target_coord[0]+1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりも上にchoiceがある
                            elif target_coord[0] > selection_coord[0]:
                                while target_coord[0]+1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                            #targetの右にchoiceを移動
                            #targetよりも右にchoiceがある
                            if target_coord[1] < selection_coord[1]:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりも左にchoiceがある
                            else:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            
                            
                            #targetを右端まで移動
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            while target_coord[1] != coord[1]+1:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            #targetを最終的に置きたいy座標まで移動
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            while target_coord[0] != coord[0]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                        #最終的におきたいx座標にある
                        elif target_coord[1] == coord[1]+1: 
                            #targetのx座標の左にchoiceを移動
                            if target_coord[1] == selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            else:
                                while target_coord[1]-1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                            #targetの左上に移動
                            #targetよりも下にchoiceがある
                            if target_coord[0] <= selection_coord[0]:
                                while target_coord[0]-1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            #targetよりも上にchoiceがある
                            else:
                                while target_coord[0]-1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                            #targetを最終的に置きたいy座標まで移動
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            while target_coord[0] != coord[0]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                #最終的に置きたい位置が右端-----------------------------------------------------------------------------------------------------------------------
                else:
                    #逆順に並んでいる
                    if target_coord[0] == coord[0] and target_coord[1] == coord[1]-1:
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                    #targetがtarget-1の下
                    elif target_coord[0] == coord[0]+1 and target_coord[1] == coord[1]:
                        #targetの左にchoice
                        #targetの左の座標に移動
                        if target_coord[1] > selection_coord[1]:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの左上に移動
                        if target_coord[0] < selection_coord[0]:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        
                        #3,4を最終的に置きたい座標となるよう移動
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                    elif target_coord[0] != coord[0]+1 or target_coord[1] != coord[1]:
                        #targetの上にchoiceがある
                        if target_coord[0] > selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの下にchoiceがある
                        else:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value) 

                        #targetの右にchoiceがある
                        if target_coord[1] < selection_coord[1]:
                            while target_coord[1] != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの左にchoiceがある
                        elif target_coord[1] > selection_coord[1]:
                            while target_coord[1] != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                        

                        #targetを右端まで移動
                        if target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                        #targetをtarget-1の下に移動
                        if target_coord[0] != coord[0]+1:
                            while target_coord[0] != coord[0]+1:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)

                        #3,4を最終的に置きたい座標となるよう移動
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)


            #targetが下端
            else:
                #最終的に起きたい位置が右2列以外-------------------------------------------------------------------------------------------------------------------
                if j != colmun-1 and j != colmun-2:
                    print("下端, 右2列以外")

                    #右2列を除いた列を並び替え
                if j != row - 1 and j != row - 2:
                    #targetが最終的におきたいx座標よりも右
                    if target_coord[1] > coord[1]:
                        #targetの上の座標にchoice(16)を置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの左上にchoice(16)を置く
                        if target_coord[1] <= selection_coord[1]:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        
                        #targetをおきたい座標のx座標と同じになるまで移動
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの上にchoiceを置く
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                    #targetが最終的におきたいx座標よりも左
                    elif target_coord[1] < coord[1]:
                        #targetの上の座標にchoice(16)を置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの右上にchoice(16)を置く
                        if target_coord[1] >= selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                        #targetを置きたい座標のx座標と同じになるまで移動
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                        #targetの上にchoic(16)を置く
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                    #targetが最終的におきたいx座標と同じ
                    else:
                        #targetの上の座標にchoice(16)を置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                        #targetの上にchoiceを置く
                        if target_coord[1] < selection_coord[1]:
                            while target_coord[1] != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)   
                        elif target_coord[1] > selection_coord[1]:
                            while target_coord[1] != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)   

                    #targetを置きたい座標のy座標になるまで移動
                    if target_coord[0] != coord[0]:
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[0] != coord[0]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                
                #最終的に置きたい位置が右から2列目-----------------------------------------------------------------------------------------------------------------------
                elif j == colmun-2:
                    print("下端, 右から2列目")

                    #targetが最終的におきたいx座標よりも左
                    if target_coord[1] < coord[1]+1:
                        #targetの上の座標にchoice(16)を置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの右上にchoice(16)を置く
                        if target_coord[1] >= selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                        #targetを置きたい座標のx座標と同じになるまで移動
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[1] != coord[1]+1:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                    #targetが最終的におきたいx座標の右と同じ
                    elif target_coord[1] == coord[1]+1:
                        #targetの上の座標にchoice(16)を置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの上にchoice(16)を移動
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                    #targetを最終的に置きたいy座標まで移動
                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                    while target_coord[0] != coord[0]:
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)

                #最終的に置きたい位置が右端-----------------------------------------------------------------------------------------------------------------------------
                else:
                    print("下端, 右端")

                    #targetが最終的におきたいx座標よりも左
                    if target_coord[1] < coord[1]:
                        #targetの上の座標にchoice(16)を置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの右上にchoice(16)を置く
                        if target_coord[1] >= selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)

                        #targetを置きたい座標のx座標と同じになるまで移動
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        while target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)

                    #targetが最終的におきたいx座標と同じ
                    elif target_coord[1] == coord[1]:
                        #targetの上の座標にchoice(16)を置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                        #targetの上にchoice(16)を移動
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                    
                    #targetを最終的に置きたいy座標まで移動
                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                    while target_coord[0] != coord[0]+1:
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
                    
                    #3,4を最終的におきたい座標となるよう移動
                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation, fixed_value)
                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation, fixed_value)
                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation, fixed_value)
                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation, fixed_value)
            
            if j != colmun-1 and j != colmun-2:
                fixed_value.append(target_value)
            elif j == colmun-1:
                fixed_value.append(target_value-1)
                fixed_value.append(target_value)

            print(puzzle_board)
            print()

    return exchange_operation, fixed_value


In [61]:
# 下2行の右4マスを除いた盤面の並び替え(たいち)
def puzzle_board_sort_bottom_left(puzzle_board, selection_times, exchange_operation, fixed_value):
    #todo: 並び替えが完了したら fixed_value に値を追加して欲しい
    
    print("がんば!")

In [62]:
# 右下4ますの並び替え(しのべー)
def puzzle_board_sort_bottom_right(puzzle_board, selection_times, exchange_operation, fixed_value):
    #todo: 並び替えが完了したら fixed_value に値を追加して欲しい
    #todo: 選択画像変更時には exchange_operation = answer_writing() を実行して欲しい
    #todo: 選択値を変更したら、変更時の座標を selectin_position に保存して欲しい

    print("がんば!")


In [63]:
# 上3つをまとめたもの
def puzzle_board_sort(puzzle_board):
    selection_times = 0                     #選択回数
    exchange_operation = ""                 #交換操作

    f = open('answer.txt', 'w')
    f.write('2\n')    
    f.close()

    fixed_value = []                        #並び替え完了の値
    
    #たくと担当
    exchange_operation, fixed_value = puzzle_board_sort_top(puzzle_board, selection_times, exchange_operation, fixed_value)
    print(exchange_operation)
    print(fix)
    #たいち担当
    # puzzle_board_sort_bottom_left(puzzle_board, selection_times, exchange_operation), fixed_value
    # #しのべー担当
    # puzzle_board_sort_bottom_right(puzzle_board, selection_times, exchange_operation, fixed_value)

# メイン関数

In [67]:
puzzle_board =  np.array([])    #現在の盤面を表す配列

#問題盤面生成
puzzle_board, colmun, row, selection_cost = puzzle_generation(puzzle_board, 'issue/4x4.txt')

print('colmun(列→)：' + str(colmun))
print('row(行↓)：' + str(row))
print('selection_cost(選択コスト)：' + str(selection_cost) + '\n')
print(puzzle_board)
print()


#並び替え
puzzle_board_sort(puzzle_board)
print(puzzle_board)

colmun(列→)：4
row(行↓)：4
selection_cost(選択コスト)：2

[[12 15 11  3]
 [ 7  1 14 13]
 [ 9  5 16  8]
 [ 2  6 10  4]]

target_num = 1----------------------------------------------------------
not下端, 右2列以外
[[ 1 12 11  3]
 [16 15 14 13]
 [ 7  9  5  8]
 [ 2  6 10  4]]

target_num = 2----------------------------------------------------------
下端, 右2列以外
[[ 1  2 12  3]
 [ 7 16 11 13]
 [ 6 14 15  8]
 [ 9 10  5  4]]

target_num = 3----------------------------------------------------------
not下端, 右から2列目
[[ 1  2 12  3]
 [ 7 16 11 13]
 [ 6 14 15  8]
 [ 9 10  5  4]]

target_num = 4----------------------------------------------------------
下端, 右端
[[ 1  2  3  4]
 [ 7 14 12 16]
 [ 6 15 13 11]
 [ 9 10  8  5]]

target_num = 5----------------------------------------------------------
下端, 右2列以外
[[ 1  2  3  4]
 [ 5  7 12 11]
 [16 14 10  8]
 [ 6  9 15 13]]

target_num = 6----------------------------------------------------------
下端, 右2列以外
[[ 1  2  3  4]
 [ 5  6  7 11]
 [ 9 16 12  8]
 [14 15 10 13]]

target_num = 7--

SystemExit: この上は盤面外、もしくは並び替えが完了しているよ!